In [ ]:
!pip install basedosdados

In [5]:
import basedosdados as bd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import plotly.express as px

billing_project_id = "onyx-basis-433918-s7"

In [ ]:
# 1. Quantos chamados foram abertos no dia 01/04/2023?
query_1 = """
    SELECT COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE DATE(data_inicio) = '2023-04-01'
"""
df_total_chamados = bd.read_sql(query_1, billing_project_id)



In [ ]:
# 2. Qual o tipo de chamado que teve mais chamados abertos no dia 01/04/2023?
query_2 = """
    SELECT tipo, COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE DATE(data_inicio) = '2023-04-01'
    GROUP BY tipo
    ORDER BY total_chamados DESC
    LIMIT 1
"""
df_tipo_mais_comum = bd.read_sql(query_2, billing_project_id)



In [ ]:
# 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?
query_3 = """
    SELECT b.nome AS nome_bairro, COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado` c
    JOIN `datario.dados_mestres.bairro` b
    ON c.id_bairro = b.id_bairro
    WHERE DATE(c.data_inicio) = '2023-04-01'
    GROUP BY nome_bairro
    ORDER BY total_chamados DESC
    LIMIT 3
"""
df_top_bairros = bd.read_sql(query_3, billing_project_id)


In [ ]:

# 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?
query_4 = """
    SELECT b.subprefeitura, COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado` c
    JOIN `datario.dados_mestres.bairro` b
    ON c.id_bairro = b.id_bairro
    WHERE DATE(c.data_inicio) = '2023-04-01'
    GROUP BY b.subprefeitura
    ORDER BY total_chamados DESC
    LIMIT 1
"""
df_top_subprefeitura = bd.read_sql(query_4, billing_project_id)



In [ ]:
# 5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros?
query_5 = """
    SELECT COUNT(*) AS total_chamados_sem_bairro
    FROM `datario.adm_central_atendimento_1746.chamado` c
    LEFT JOIN `datario.dados_mestres.bairro` b
    ON c.id_bairro = b.id_bairro
    WHERE DATE(c.data_inicio) = '2023-04-01' AND b.id_bairro IS NULL
"""
df_chamados_sem_bairro = bd.read_sql(query_5, billing_project_id)



In [ ]:
# 6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023?
query_6 = """
    SELECT COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE subtipo = 'Perturbação do sossego'
    AND DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
"""
df_total_perturbacao = bd.read_sql(query_6, billing_project_id)



In [ ]:
# 7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos.
query_7 = """
    SELECT c.*
    FROM `datario.adm_central_atendimento_1746.chamado` c
    JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
    ON DATE(c.data_inicio) BETWEEN e.data_inicial AND e.data_final
    WHERE c.subtipo = 'Perturbação do sossego'
"""
df_chamados_eventos = bd.read_sql(query_7, billing_project_id)



In [ ]:
# 8. Quantos chamados desse subtipo foram abertos em cada evento?
query_8 = """
    SELECT e.evento, COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado` c
    JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
    ON DATE(c.data_inicio) BETWEEN e.data_inicial AND e.data_final
    WHERE c.subtipo = 'Perturbação do sossego'
    GROUP BY e.evento
"""
df_chamados_por_evento = bd.read_sql(query_8, billing_project_id)



In [ ]:
# 9. Qual evento teve a maior média diária de chamados abertos desse subtipo?
query_9 = """
    SELECT e.evento, COUNT(*) / COUNT(DISTINCT DATE(c.data_inicio)) AS media_diaria_chamados
    FROM `datario.adm_central_atendimento_1746.chamado` c
    JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
    ON DATE(c.data_inicio) BETWEEN e.data_inicial AND e.data_final
    WHERE c.subtipo = 'Perturbação do sossego'
    GROUP BY e.evento
    ORDER BY media_diaria_chamados DESC
    LIMIT 1
"""
df_evento_mais_chamados = bd.read_sql(query_9, billing_project_id)



In [ ]:
# 10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos e a média diária de chamados abertos desse subtipo considerando todo o período.
query_10_eventos = """
    SELECT e.evento, COUNT(*) / COUNT(DISTINCT DATE(c.data_inicio)) AS media_diaria_evento
    FROM `datario.adm_central_atendimento_1746.chamado` c
    JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
    ON DATE(c.data_inicio) BETWEEN e.data_inicial AND e.data_final
    WHERE c.subtipo = 'Perturbação do sossego'
    GROUP BY e.evento
"""
df_media_diaria_eventos = bd.read_sql(query_10_eventos, billing_project_id)

query_10_geral = """
    SELECT COUNT(*) / COUNT(DISTINCT DATE(data_inicio)) AS media_diaria_geral
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE subtipo = 'Perturbação do sossego'
    AND DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
"""
df_media_diaria_geral = bd.read_sql(query_10_geral, billing_project_id)



In [ ]:
# Resultados
print(f"Total de chamados em 01/04/2023: {df_total_chamados.iloc[0, 0]}")
print(f"Tipo de chamado mais comum em 01/04/2023: {df_tipo_mais_comum.iloc[0, 0]} com {df_tipo_mais_comum.iloc[0, 1]} chamados")
print("Top 3 bairros com mais chamados em 01/04/2023:")
print(df_top_bairros)
print(f"Subprefeitura com mais chamados em 01/04/2023: {df_top_subprefeitura.iloc[0, 0]}")
print(f"Chamados em 01/04/2023 sem bairro associado: {df_chamados_sem_bairro.iloc[0, 0]}")
print(f"Total de chamados por 'Perturbação do sossego' entre 01/01/2022 e 31/12/2023: {df_total_perturbacao.iloc[0, 0]}")
print(f"Total de chamados por 'Perturbação do sossego' durante os eventos: {df_chamados_eventos.shape[0]}")
print("Chamados por 'Perturbação do sossego' por evento:")
print(df_chamados_por_evento)
print(f"Evento com maior média diária de chamados: {df_evento_mais_chamados.iloc[0, 0]}")
print("Médias diárias durante os eventos específicos:")
print(df_media_diaria_eventos)
print(f"Média diária geral: {df_media_diaria_geral.iloc[0, 0]}")


In [ ]:
# Consulta para obter chamados ao longo do tempo
query_chamados_tempo = """
    SELECT DATE(data_inicio) AS data, COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE subtipo = 'Perturbação do sossego'
    AND DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
    GROUP BY data
    ORDER BY data
"""
df_chamados_tempo = bd.read_sql(query_chamados_tempo, billing_project_id)

# Gráfico de Linha
plt.figure(figsize=(14, 7))
plt.plot(df_chamados_tempo['data'], df_chamados_tempo['total_chamados'], marker='o', linestyle='-')
plt.title('Distribuição Diária de Chamados de Perturbação do Sossego (2022-2023)')
plt.xlabel('Data')
plt.ylabel('Número de Chamados')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
query_chamados_por_bairro = """
    SELECT b.nome AS bairro,
           COUNT(*) AS total_chamadas
    FROM `datario.adm_central_atendimento_1746.chamado` c
    JOIN `datario.dados_mestres.bairro` b
    ON c.id_bairro = b.id_bairro
    GROUP BY b.nome
"""
# Executar a consulta para contar chamadas por bairro
df_chamados_por_bairro = bd.read_sql(query_chamados_por_bairro, billing_project_id)

# Consultar os dados dos bairros com geometria
query_bairros = """
    SELECT nome AS bairro,
           geometry_wkt
    FROM `datario.dados_mestres.bairro`
"""
df_bairros = bd.read_sql(query_bairros, billing_project_id)

In [ ]:
# Converter a coluna de geometria de WKT para o tipo geométrico
df_bairros['geometry_wkt'] = gpd.GeoSeries.from_wkt(df_bairros['geometry_wkt'])

# Criar um GeoDataFrame com a geometria dos bairros
gdf_bairros = gpd.GeoDataFrame(df_bairros, geometry='geometry_wkt')

# Juntar os dados de chamadas com o GeoDataFrame dos bairros
gdf_bairros = gdf_bairros.merge(df_chamados_por_bairro, on='bairro', how='left')

# Preencher valores nulos (caso haja bairros sem chamadas) com zero
gdf_bairros['total_chamadas'].fillna(0, inplace=True)
gdf_bairros.head()

In [ ]:
# Converter a coluna 'total_chamadas' para numérico, forçando erros a se tornarem NaN
gdf_bairros['total_chamadas'] = pd.to_numeric(gdf_bairros['total_chamadas'], errors='coerce')

# Preencher valores NaN com 0 ou outro valor apropriado
gdf_bairros['total_chamadas'] = gdf_bairros['total_chamadas'].fillna(0)

# Garantir que a coluna é do tipo float
gdf_bairros['total_chamadas'] = gdf_bairros['total_chamadas'].astype(float)

# Criar a figura e o eixo
fig, ax = plt.subplots(figsize=(70, 70))  # Ajuste o tamanho da figura conforme necessário

# Definir o cmap e os intervalos de chamadas
cmap = plt.get_cmap('OrRd')  # Escolha uma paleta de cores
norm = plt.Normalize(vmin=gdf_bairros['total_chamadas'].min(), vmax=gdf_bairros['total_chamadas'].max())

# Plotar o GeoDataFrame dos bairros usando a coluna 'total_chamadas' para colorir
gdf_bairros.plot(column='total_chamadas', ax=ax, edgecolor='k', cmap=cmap, alpha=0.7, norm=norm, legend=False)
# Adicionar os nomes dos bairros
for idx, row in gdf_bairros.iterrows():
    center = row['geometry_wkt'].representative_point()
    ax.text(center.x, center.y, row['bairro'], fontsize=10, ha='center', color='black')

# Adicionar título e ajustar os eixos
ax.set_title('Chamadas Totais por Bairro', fontsize=40)
ax.set_xlabel('Longitude', fontsize=30)
ax.set_ylabel('Latitude', fontsize=30)
ax.tick_params(labelsize=30)

# Ajustar o aspecto do mapa
ax.set_aspect('equal')

# Adicionar a legenda
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, orientation='vertical', fraction=0.02, pad=0.04)

# Ajustar o tamanho da legenda
cbar.set_label('Total de Chamadas', fontsize=30)
cbar.ax.tick_params(labelsize=30)  # Reduzir o tamanho das marcas na barra de cores

# Mostrar o mapa
plt.show()


In [ ]:
# Consulta para obter chamados por hora
query_chamados_hora = """
    SELECT EXTRACT(HOUR FROM data_inicio) AS hora, COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE subtipo IS NOT NULL
    GROUP BY hora
    ORDER BY hora
"""
df_chamados_hora = bd.read_sql(query_chamados_hora, billing_project_id)

# Gráfico de Barras
plt.figure(figsize=(12, 7))
sns.barplot(x='hora', y='total_chamados', data=df_chamados_hora, palette='viridis')
plt.title('Número de Chamados por Hora do Dia')
plt.xlabel('Hora do Dia')
plt.ylabel('Número de Chamados')
plt.show()


In [ ]:
# Consulta para obter chamados por hora
query_chamados_hora = """
    SELECT EXTRACT(HOUR FROM data_inicio) AS hora, COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE subtipo = 'Perturbação do sossego'
    GROUP BY hora
    ORDER BY hora
"""
df_chamados_hora = bd.read_sql(query_chamados_hora, billing_project_id)

# Gráfico de Barras
plt.figure(figsize=(12, 7))
sns.barplot(x='hora', y='total_chamados', data=df_chamados_hora, palette='viridis')
plt.title('Número de Chamados de Perturbação do Sossego por Hora do Dia')
plt.xlabel('Hora do Dia')
plt.ylabel('Número de Chamados')
plt.show()


In [ ]:
# Consulta para obter distribuição de tipos de chamados em um dia específico
query_tipo_chamado = """
    SELECT tipo, COUNT(*) AS total_chamados
    FROM `datario.adm_central_atendimento_1746.chamado`
    WHERE DATE(data_inicio) = '2023-04-01'
    GROUP BY tipo
    ORDER BY total_chamados DESC
"""
df_tipo_chamado = bd.read_sql(query_tipo_chamado, billing_project_id)

# Gráfico de Barras
plt.figure(figsize=(12, 8))
sns.barplot(x=df_tipo_chamado['total_chamados'], y=df_tipo_chamado['tipo'], palette='Set2')
plt.title('Distribuição de Tipos de Chamados em 01/04/2023')
plt.xlabel('Total de Chamados')
plt.ylabel('Tipo de Chamado')
plt.show()


In [ ]:
# Definir a consulta SQL para carregar os dados
query = """
SELECT
    id_chamado,
    data_inicio,
    data_fim,
    id_bairro,
    id_territorialidade,
    id_unidade_organizacional,
    nome_unidade_organizacional,
    tipo,
    status,
    tempo_prazo,
    prazo_unidade,
    prazo_tipo,
    dentro_prazo
FROM
    `datario.adm_central_atendimento_1746.chamado`
WHERE
    DATE(data_inicio) = '2023-04-01'
"""

# Executar a consulta e carregar os dados em um DataFrame
df_chamados = bd.read_sql(query, billing_project_id)

# Converter as colunas de data para o tipo datetime
df_chamados['data_inicio'] = pd.to_datetime(df_chamados['data_inicio'])
df_chamados['data_fim'] = pd.to_datetime(df_chamados['data_fim'])

# Calcular o tempo de resolução em dias
df_chamados['tempo_resolucao'] = (df_chamados['data_fim'] - df_chamados['data_inicio']).dt.days

# Preencher valores NaN na coluna de tempo de resolução com 0 ou outro valor apropriado
df_chamados['tempo_resolucao'] = df_chamados['tempo_resolucao'].fillna(0)

# Garantir que a coluna é do tipo float
df_chamados['tempo_resolucao'] = df_chamados['tempo_resolucao'].astype(float)


In [25]:
# Tempo médio de resolução por tipo de chamado
tempo_medio_tipo = df_chamados.groupby('tipo')['tempo_resolucao'].mean().reset_index()

# Tempo médio de resolução por subprefeitura
tempo_medio_subprefeitura = df_chamados.groupby('id_territorialidade')['tempo_resolucao'].mean().reset_index()

In [ ]:
# Gráfico de tempo médio de resolução por tipo de chamado
plt.figure(figsize=(14, 7))
plt.bar(tempo_medio_tipo['tipo'], tempo_medio_tipo['tempo_resolucao'], color='skyblue')
plt.xlabel('Tipo de Chamado')
plt.ylabel('Tempo Médio de Resolução (dias)')
plt.title('Tempo Médio de Resolução por Tipo de Chamado')
plt.xticks(rotation=45, ha='right')
plt.show()

# Gráfico de tempo médio de resolução por subprefeitura
plt.figure(figsize=(14, 7))
plt.bar(tempo_medio_subprefeitura['id_territorialidade'], tempo_medio_subprefeitura['tempo_resolucao'], color='lightcoral')
plt.xlabel('Subprefeitura')
plt.ylabel('Tempo Médio de Resolução (dias)')
plt.title('Tempo Médio de Resolução por Subprefeitura')
plt.xticks(rotation=45, ha='right')
plt.show()
